In [1]:
import json
import numpy as np
from pathlib import Path

In [8]:
data = {}
case = ""
prop = "r"

print("Processing Native Layout data...\n")

Processing Native Layout data...



In [9]:
# Define all IBM devices with their data files
ibm_devices = {
    # Heron processors (127q) - use regular 127_HE.npy for all
    "ibm_brisbane": ("Data/ibm_brisbane/127_HE.npy", 1, None),
    "ibm_brussels": ("Data/ibm_brussels/127_HE.npy", 1, None),
    "ibm_kyiv": ("Data/ibm_kyiv/127_HE.npy", 1, None),
    "ibm_kyoto": ("Data/ibm_kyoto/127_HE.npy", 1, None),
    "ibm_nazca": ("Data/ibm_nazca/127_HE.npy", 1, None),
    "ibm_osaka": ("Data/ibm_osaka/127_HE.npy", 1, None),
    "ibm_sherbrooke": ("Data/ibm_sherbrooke/127_HE.npy", 1, None),
    "ibm_strasbourg": ("Data/ibm_strasbourg/127_HE.npy", 1, None),
    
    # Heron processors (133q)
    "ibm_torino-v0": ("Data/ibm_torino/133_HE.npy", 1, None),
    "ibm_torino-v1": ("Data/ibm_torino/133_HE_v1.npy", 1, None),
    "ibm_torino-f": ("Data/ibm_torino/133_HE_fractional.npy", 0.75, None),
    
    # Eagle processors (156q)
    "ibm_fez": ("Data/ibm_fez/156_HE.npy", 1, None),
    "ibm_fez-f": ("Data/ibm_fez/156_HE_fractional.npy", 0.75, None),
    "ibm_marrakesh-f": ("Data/ibm_marrakesh/156_HE_fractional.npy", 0.75, None),
    "ibm_aachen": ("Data/ibm_aachen/156_HE.npy", 0.75, "slice"),  # ps[:-2]
    "ibm_aachen-f": ("Data/ibm_aachen/156_HE_fractional.npy", 0.75, None),
    "ibm_kingston": ("Data/ibm_kingston/156_HE.npy", 0.75, "slice"),  # ps[:-2]
    "ibm_kingston-f": ("Data/ibm_kingston/156_HE_fractional.npy", 0.75, None),
    "ibm_boston-f": ("Data/ibm_boston/156_HEw1.npy", 0.75, None),
    "ibm_pittsburgh": ("Data/ibm_pittsburgh/156_HE.npy", 0.75, None),
    "ibm_pittsburgh-f": ("Data/ibm_pittsburgh/156_HEfrac.npy", 0.75, None),
}

for backend_name, (file_path, delta, slice_mode) in ibm_devices.items():
    try:
        results = np.load(file_path, allow_pickle=True).item()
        ps = results["ps"]
        
        # Apply slicing if needed (for aachen and kingston non-fractional)
        if slice_mode == "slice":
            ps = ps[:-2]
            rs = [results[f"postprocessing{case}"][delta][p][prop] for p in ps]
        else:
            rs = [results[f"postprocessing{case}"][delta][p][prop] for p in ps]
        
        # Check if random data exists
        has_random = f"random{case}" in results
        random_r = results.get(f"random{case}", {}).get(prop, 0.5) if has_random else 0.5
        
        # Get qubit count from file name or results
        if "127" in file_path:
            qubits = 127
        elif "133" in file_path:
            qubits = 133
        elif "156" in file_path:
            qubits = 156
        else:
            qubits = results.get("nq", 0)
        
        data[backend_name] = {
            "qubits": qubits,
            "p_values": ps,
            "r_values": [round(r, 3) for r in rs],
            "max_r": round(max(rs), 3),
            "optimal_p": int(ps[np.argmax(rs)]),
            "has_random": has_random,
            "random_r": round(random_r, 3)
        }
        print(f"  {backend_name}: {qubits}q, max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
        
    except Exception as e:
        print(f"  Warning: Could not load {backend_name}: {e}")


  ibm_brussels: 127q, max_r = 0.644 at p = 4
  ibm_kyiv: 127q, max_r = 0.706 at p = 4
  ibm_kyoto: 127q, max_r = 0.636 at p = 4
  ibm_osaka: 127q, max_r = 0.617 at p = 5
  ibm_strasbourg: 127q, max_r = 0.628 at p = 4
  ibm_torino-v0: 133q, max_r = 0.724 at p = 6
  ibm_torino-v1: 133q, max_r = 0.746 at p = 6
  ibm_torino-f: 133q, max_r = 0.768 at p = 13
  ibm_fez: 156q, max_r = 0.751 at p = 6
  ibm_fez-f: 156q, max_r = 0.771 at p = 10
  ibm_aachen: 156q, max_r = 0.767 at p = 9
  ibm_aachen-f: 156q, max_r = 0.806 at p = 15
  ibm_kingston: 156q, max_r = 0.731 at p = 8
  ibm_kingston-f: 156q, max_r = 0.815 at p = 15
  ibm_boston-f: 156q, max_r = 0.835 at p = 25
  ibm_pittsburgh: 156q, max_r = 0.782 at p = 15
  ibm_pittsburgh-f: 156q, max_r = 0.834 at p = 20


In [4]:
# Add IQM devices
print("\nProcessing IQM devices...")

iqm_devices = {
    "iqm_garnet": ("Data/iqm_garnet/20_NL.npy", 1, 0),
    "iqm_emerald": ("Data/iqm_emerald/54_HE.npy", 0.75, None),
}

for backend_name, (file_path, delta, section) in iqm_devices.items():
    try:
        results = np.load(file_path, allow_pickle=True).item()
        ps = results["ps"]
        
        if section is not None:
            rs = [results[f"postprocessing{case}"][delta][p][section][prop] for p in ps]
        else:
            rs = [results[f"postprocessing{case}"][delta][p][prop] for p in ps]
        
        # Get qubit count
        if "20" in file_path:
            qubits = 20
        elif "54" in file_path:
            qubits = 54
        else:
            qubits = results.get("nq", 0)
        
        data[backend_name] = {
            "qubits": qubits,
            "p_values": ps,
            "r_values": [round(r, 3) for r in rs],
            "max_r": round(max(rs), 3),
            "optimal_p": int(ps[np.argmax(rs)]),
            "has_random": False,
            "random_r": 0.5
        }
        print(f"  {backend_name}: {qubits}q, max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
        
    except Exception as e:
        print(f"  Warning: Could not load {backend_name}: {e}")


Processing IQM devices...
  iqm_garnet: 20q, max_r = 0.664 at p = 4
  iqm_emerald: 54q, max_r = 0.651 at p = 4


In [5]:
# Add Rigetti devices
print("\nProcessing Rigetti devices...")

try:
    results = np.load("Data/rigetti_ankaa_3/82_NL.npy", allow_pickle=True).item()
    ps = results["ps"]
    rs = [results[f"postprocessing{case}"][1][p][0][prop] for p in ps]
    
    data["rigetti_ankaa_3"] = {
        "qubits": 82,
        "p_values": ps,
        "r_values": [round(r, 3) for r in rs],
        "max_r": round(max(rs), 3),
        "optimal_p": int(ps[np.argmax(rs)]),
        "has_random": False,
        "random_r": 0.5
    }
    print(f"  rigetti_ankaa_3: 82q, max_r = {round(max(rs), 3)} at p = {ps[np.argmax(rs)]}")
except Exception as e:
    print(f"  Warning: Could not load rigetti_ankaa_3: {e}")


Processing Rigetti devices...
  rigetti_ankaa_3: 82q, max_r = 0.48 at p = 3


In [6]:
# Save to JSON
output_path = Path("Data/native_layout_processed.json")
with open(output_path, 'w') as f:
    json.dump(data, f, indent=2)

print(f"\n✓ Successfully saved data to {output_path}")
print(f"  Total backends: {len(data)}")
print(f"  IBM: {len([k for k in data.keys() if k.startswith('ibm_')])}")
print(f"  IQM: {len([k for k in data.keys() if k.startswith('iqm_')])}")
print(f"  Rigetti: {len([k for k in data.keys() if k.startswith('rigetti_')])}")


✓ Successfully saved data to Data/native_layout_processed.json
  Total backends: 20
  IBM: 17
  IQM: 2
  Rigetti: 1


In [ ]:
# Check what's in the brisbane regular file
results_bris = np.load("Data/ibm_brisbane/127_HE.npy", allow_pickle=True).item()
print("Brisbane 127_HE.npy keys:", list(results_bris.keys())[:10])
print("File size check - this is a very small file (6992 bytes), likely incomplete")


AttributeError: 'Model' object has no attribute '_var_scope'